# Blinks

In this notebook, the analysis on the blinks is done.

First, all the dependencies are imported and some functions are defined.

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import datetime
import tensorflow as tf
import os
import warnings

from keras import backend as K

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dropout, Flatten, Dense

from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight

from sys import platform

#all the functions are in this module
from EEGAnalysis import *

warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
def newSession(GPU=0, CPU=8, randomSeed = 8):
    #delete previous model and set up session. Also resets the random seed.
    np.random.seed(randomSeed)
    tf.keras.backend.clear_session()
    config = tf.ConfigProto(device_count={'GPU':GPU, 'CPU':CPU})
    config.gpu_options.allow_growth = True
    config.gpu_options.per_process_gpu_memory_fraction = 0.9
    tf.keras.backend.set_session(tf.Session(config=config))
    gc.collect()

Here the file is imported and processed.

In [3]:
#import the file
filename = 'EEGdata/blinks_2019.03.25_14.56.23.edf'
rate = 256

#readEDF-function in python module EEGAnalysis
EEGData, markerData = readEDF(filename, CQ = True, markers = True, motion = False, powerLineHumFilter = True)
xTrain, yTrain, xTest, yTest = None, None, None, None

In [4]:
classes = ['blank', 'blink']
events = createEvents(markerData, np.array([4, 5]))

#create Epochs
tmin = -0.2   # set time around events to create Epochs
tmax = 0.8

#parameters for the creation of the Epochs
CQ = True
normalize = True

In [5]:
(xTrain, yTrain),(xTest, yTest) = createEpochs(EEGData, events, tmin, tmax, CQ = CQ, normalize = normalize, verbose = False)

Done making training/test-sets, shapes:  (99, 257, 14) (99, 2) (12, 257, 14) (12, 2)


Now we're defining and training the neural network

In [6]:
#hyperparameters
verbose = 0
batchSizeGlobal = 32
epochs = 30 #number of times the network is trained on the entire set.
loadPreviousModels = False #loads the checkpoints
run = '0300-Blinks-'
LR = 1e-3
loss = 'binary_crossentropy'
metrics = ['accuracy', f1]
patience = 10
min_delta = 1e-5

#prevent naive model
classWeight = {}
for i in range(len(classes)):
    classWeight[i] = unpackClasses(yTrain).tolist().count(i)

newSession()

print(xTrain.shape[0], 'train samples')
print(xTest.shape[0], 'test samples')

#build model
initializer = None

model = Sequential()
model.add(Flatten(input_shape= (xTrain.shape[1:])))
model.add(Dense(64, activation='relu',
                kernel_initializer = initializer))
model.add(Dropout(rate = 0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(rate = 0.3))
model.add(Dense(len(yTest[0]), activation='sigmoid'))
model.summary()

optimizer = tf.keras.optimizers.Adam(lr=LR)#, decay=1.5)#rho=rho, epsilon=epsilon,
model.compile(optimizer=optimizer, loss=loss,  metrics=metrics)

#make filepaths
networkname = run + makeNameModel(model)
time = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
logDir = './blinks/logs/' + networkname + time
modelFile = './blinks/logs/' + networkname + '.hdf5'
if platform.count('Windows'):
    logDir = os.getcwd() + logDir
    modelFile = os.getcwd() + modelFile
    if not loadPreviousModels:
        os.makedirs(logDir)

if loadPreviousModels:
    model.load_weights(modelFile)

# prepare callbacks and checkpoint learning
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=logDir, histogram_freq=0,
                            write_graph=True, write_images=False, write_grads = True, update_freq = 'epoch')
checkpoint = tf.keras.callbacks.ModelCheckpoint(modelFile, monitor='val_loss', mode = 'min', period=5, save_best_only=False)
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode = 'min', patience=patience, min_delta = min_delta)
histories = Histories()

print(networkname)

#batch size
batch_size = 32

model.fit(xTrain, yTrain, epochs=epochs, batch_size=batch_size, verbose = verbose,
          callbacks = [tensorboard, checkpoint, earlyStopping, histories],
          validation_data=(xTest, yTest), class_weight=classWeight)

trainResult = np.array([histories.loss, histories.acc, histories.f1])
testResult = np.array([histories.val_loss, histories.val_acc, histories.val_f1])

score = model.evaluate(xTest, yTest, verbose=1)

y_pred = model.predict_classes(xTest)
y_true = unpackClasses(yTest)
conf_matrix_test = confusion_matrix(y_true, y_pred)
dispMat(conf_matrix_test, names = classes)

temp = 0
for i in model.metrics_names:
    print('Test', str(i), ':', round(score[temp], 2))
    temp += 1

99 train samples
12 test samples
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 3598)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                230336    
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
______________________________________________________

,blank,blink
blank,1,2
blink,0,9


Test loss : 1.55
Test acc : 0.83
Test f1 : 0.83


In [7]:
#log the results in a .npy file
logDir = os.getcwd() + '/blinks/logsNpy'
trainFile = logDir + '/' + networkname + '-train'
testFile = logDir + '/' +  networkname + '-test'

try:
    np.save(trainFile, trainResult)
    np.save(testFile, testResult)
except:
    os.makedirs(logDir)
    np.save(trainFile, trainResult)
    np.save(testFile, testResult)